In [1]:
import numpy as np
import tensorflow as tf
import datetime
import matplotlib.pyplot as plt
import pickle
import matplotlib.image as mpimg

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# load the data
data_file = open('cifar_10_tf_train_test.pkl', 'rb')
train_data, train_label, test_data, test_label = pickle.load(data_file, encoding='latin1')
print(train_data.shape)
data_file.close()
train_label = np.array(train_label)
test_label = np.array(test_label)

(50000, 32, 32, 3)


In [3]:
hm_train_data, dim1, dim2, hm_channel = train_data.shape
hm_test_data, _, _, _ = test_data.shape
hm_class = 10

batch_size = 100
max_epoch = 50
eval_interval = 300

# pre-process the data, cast to float32, scaling, and normalize
train_data = train_data.astype(np.float32) / 255.
train_label = train_label.astype(np.int64)
test_data = test_data.astype(np.float32) / 255.
test_label = test_label.astype(np.int64)

train_data -= np.mean(train_data, axis=(1, 2, 3), keepdims=True)
test_data -= np.mean(test_data, axis=(1, 2, 3), keepdims=True)
test_acc = np.zeros((0, 1))
train_acc = np.zeros((0, 1))
loss = np.zeros((0, 1))

In [4]:
x = tf.placeholder(shape=[None, 32, 32, 3], dtype=tf.float32)
y = tf.placeholder(shape=[None], dtype=tf.int64)
keep_prob = tf.placeholder(dtype=tf.float32)

In [5]:
def weight_init(name, shape):
    initial = tf.truncated_normal(shape, stddev=0.01) / 100.
    return tf.Variable(initial)


def bias_init(name, shape):
    initial = tf.constant(0.001, shape=shape)
    return tf.Variable(initial)


def conv2d(x, w):
    return tf.nn.conv2d(x, w, strides=[1, 1, 1, 1], padding='VALID')


def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')


def avg_pool_2x2(x):
    return tf.nn.avg_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')


def drop_out(h, portion):
    return tf.nn.dropout(h, portion)

def predict(h_conv3, b_fc1, w_fc1, portion):
    h_conv3_drop = drop_out(h_conv3, portion)
    fc1 = tf.reshape(h_conv3_drop, [-1, 3 * 3 * 64])
    y_out = tf.matmul(fc1, w_fc1) + b_fc1
    return tf.argmax(y_out,1)

In [6]:
# Forward propagation, 1st stage
w_conv1 = weight_init('weight_conv1', [5, 5, 3, 32])
b_conv1 = bias_init('bias_conv1', [28, 28, 32])
image = tf.reshape(x, [-1, 32, 32, 3])

h_conv1 = tf.nn.relu(conv2d(image, w_conv1) + b_conv1)
h_pool1 = avg_pool_2x2(h_conv1)

# 2nd stage
w_conv2 = weight_init('weight_conv2', [5, 5, 32, 32])
b_conv2 = bias_init('bias_conv2', [10, 10, 32])

h_conv2 = tf.nn.relu(conv2d(h_pool1, w_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# 3rd stage
w_conv3 = weight_init('weight_conv3', [3, 3, 32, 64])
b_conv3 = bias_init('bias_conv3', [3, 3, 64])

h_conv3 = tf.nn.relu(conv2d(h_pool2, w_conv3) + b_conv3)

#h_pool3 = max_pool_2x2(h_conv3)

# drop out 50% during training

h_conv3_drop = drop_out(h_conv3, keep_prob)



# fully connected layer
fc1 = tf.reshape(h_conv3_drop, [-1, 3 * 3 * 64])
w_fc1 = weight_init('weight_fc1', [3 * 3 * 64, hm_class])
b_fc1 = bias_init('bias_fc1', [hm_class])

# output layer
y_out = tf.matmul(fc1, w_fc1) + b_fc1

In [7]:
cross_entropy = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y_out, labels=y))
train_step = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cross_entropy)
#train_step = tf.train.RMSPropOptimizer(learning_rate=0.002, decay=0.6).minimize(cross_entropy)
predict_op = predict(h_conv3, b_fc1, w_fc1, 1)
correct_prediction = tf.equal(tf.argmax(y_out, 1), y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Save the model
tf.get_collection('validation_nodes')

# Add opts to the collection
tf.add_to_collection('validation_nodes', x)
tf.add_to_collection('validation_nodes', y)
tf.add_to_collection('validation_nodes', predict_op)

In [8]:
# start training and eval accuracy
# with tf.Session() as sess:
saver = tf.train.Saver()
sess = tf.Session()
#with tf.Session() as sess:
sess.run(tf.global_variables_initializer())

for epoch in np.arange(max_epoch):
    start = datetime.datetime.now()
    re_order = np.random.permutation(hm_train_data)
    

    for i in np.arange(int(hm_train_data / batch_size)):
        batch_x = train_data[re_order[i * batch_size:(i + 1) * batch_size], :, :, :]
        batch_y = train_label[re_order[i * batch_size:(i + 1) * batch_size]]

        # evaluate the training performance
        if i % eval_interval == 0:
            ce = sess.run(cross_entropy, feed_dict={x: batch_x, y:batch_y, keep_prob:1})
            train_accuracy = sess.run(accuracy, feed_dict={x: batch_x, y:batch_y, keep_prob:1})
            test_accuracy = sess.run(accuracy, feed_dict={x: test_data, y: test_label, keep_prob:1})
            train_acc = np.vstack((train_acc, train_accuracy))
            test_acc = np.vstack((test_acc, test_accuracy))
            loss = np.vstack((loss, ce))
            print("epoch: %d step: %d, training accuracy: %g, testing accuracy: %g, loss: %g" %
                            (epoch, i, train_accuracy, test_accuracy, ce))

        sess.run(train_step, feed_dict={x: batch_x, y: batch_y, keep_prob:0.5})

    dur = datetime.datetime.now() - start
    print('Epoch', epoch, 'training done, runtime', dur)

#train_accuracy = sess.run(accuracy, feed_dict={x: train_data, y: train_label, keep_prob:1})
test_accuracy = sess.run(accuracy, feed_dict={x: test_data, y: test_label, keep_prob:1})
print("Training finished! Final training accuracy: %g, testing accuracy: %g" %
                (np.mean(train_acc[-int(hm_train_data/batch_size):, :]), test_accuracy))
#train_acc = np.vstack((train_acc, train_accuracy))
test_acc = np.vstack((test_acc, test_accuracy))


epoch: 0 step: 0, training accuracy: 0.06, testing accuracy: 0.1026, loss: 2.30259
epoch: 0 step: 300, training accuracy: 0.37, testing accuracy: 0.3412, loss: 1.73188
Epoch 0 training done, runtime 0:00:06.492011
epoch: 1 step: 0, training accuracy: 0.46, testing accuracy: 0.444, loss: 1.65926
epoch: 1 step: 300, training accuracy: 0.47, testing accuracy: 0.498, loss: 1.3384
Epoch 1 training done, runtime 0:00:04.686205
epoch: 2 step: 0, training accuracy: 0.5, testing accuracy: 0.529, loss: 1.46833
epoch: 2 step: 300, training accuracy: 0.6, testing accuracy: 0.5498, loss: 1.05433
Epoch 2 training done, runtime 0:00:04.614797
epoch: 3 step: 0, training accuracy: 0.53, testing accuracy: 0.565, loss: 1.18474
epoch: 3 step: 300, training accuracy: 0.54, testing accuracy: 0.5808, loss: 1.24955
Epoch 3 training done, runtime 0:00:04.679670
epoch: 4 step: 0, training accuracy: 0.6, testing accuracy: 0.5964, loss: 1.20846
epoch: 4 step: 300, training accuracy: 0.57, testing accuracy: 0.6044

Epoch 37 training done, runtime 0:00:04.708333
epoch: 38 step: 0, training accuracy: 0.86, testing accuracy: 0.7014, loss: 0.383965
epoch: 38 step: 300, training accuracy: 0.86, testing accuracy: 0.7094, loss: 0.362346
Epoch 38 training done, runtime 0:00:04.717040
epoch: 39 step: 0, training accuracy: 0.91, testing accuracy: 0.7098, loss: 0.231378
epoch: 39 step: 300, training accuracy: 0.92, testing accuracy: 0.7108, loss: 0.311639
Epoch 39 training done, runtime 0:00:04.666042
epoch: 40 step: 0, training accuracy: 0.92, testing accuracy: 0.705, loss: 0.233637
epoch: 40 step: 300, training accuracy: 0.78, testing accuracy: 0.7082, loss: 0.501941
Epoch 40 training done, runtime 0:00:04.651846
epoch: 41 step: 0, training accuracy: 0.85, testing accuracy: 0.7102, loss: 0.37816
epoch: 41 step: 300, training accuracy: 0.82, testing accuracy: 0.6992, loss: 0.463761
Epoch 41 training done, runtime 0:00:04.630563
epoch: 42 step: 0, training accuracy: 0.86, testing accuracy: 0.7118, loss: 0.3

In [10]:
save_path = saver.save(sess, './my_model')